In [ ]:
!pip install pymorphy2
!pip install razdel

In [1]:
import pickle
import numpy as np
import pandas as pd
import pymorphy2
from razdel import sentenize
from razdel import tokenize
import torch
from transformers import AutoTokenizer, AutoModel
import re
import gc
import tqdm

In [5]:
with open("ru_vocab_single_words.txt", encoding='utf-8') as f:
    lst = eval(f.read())

In [12]:
len(lst)

73834

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruRoberta-large")
model = AutoModel.from_pretrained("sberbank-ai/ruRoberta-large", output_hidden_states=True)
model.eval()

In [ ]:
with open("/kaggle/input/wordstobuild/vocab.pkl", 'rb') as f:
    df = pickle.load(f)

In [ ]:
pm = pymorphy2.MorphAnalyzer()

In [ ]:
contexts = {}

In [ ]:
from numpy.random import default_rng
rng = default_rng()

In [ ]:
model.to('cuda')

max_len = 90
n_contexts = 50
i = 1

for key, value in tqdm.tqdm(df.items()):
#     print(f'Iteration {i} / {len(ws_full)}')
    i += 1    
    vector = [] # здесь будут храниться эмбеддинги, то есть его размер будет (n_contexts, 312)
    n = 0
    size = n_contexts if n_contexts <= len(value) else len(value)
    indices = rng.choice(len(value), size, replace=False)
    batch = [i[0] for i in value]
    words = [list(tokenize(i[0].lower()))[i[1]] for i in value]
    with torch.no_grad():
        d = tokenizer(batch, return_offsets_mapping=True, return_tensors='pt', return_attention_mask=False,
                          return_token_type_ids=False, padding=True, truncation=True, max_length=90)
        offset_mapping = d.pop('offset_mapping')
        input_ids = d['input_ids'].to('cuda')
        sent_vec = model(input_ids).hidden_states[5].to('cpu')
        for k in range(sent_vec.shape[0]):
            start = np.where(offset_mapping[k, :, 0] == words[k].start)[0]
            if (start.shape[0] == 0):
                continue
            start = start[0]
            stop = np.where(offset_mapping[k, :, 1] == words[k].stop)[0]
            if (stop.shape[0] == 0):
                continue
            stop = stop[0]
            vector.append(sent_vec[k, start:stop+1, :].mean(axis=0))
        if (len(vector) == 0):
            continue
        vector = torch.stack(vector)b
        
        contexts[key] = vector.mean(axis=0)